# Exercise 4b - Side by Side Comparisons
In this exercise you'll evaluate the performance of two RAG applications side-by-side.

In [ ]:
%load_ext dotenv
%dotenv ../../.env

Similar to exercise 4a, first the config files are loaded to configure our RAG applications. In this exercise, we configure two RAG applications which we'll compare side by side.

In [ ]:
import os
import json
from pathlib import Path

current_dir_path: Path = Path(".")

config_1_file_name: str = "config-4a.json"
config_1_path: Path = current_dir_path / config_1_file_name

config_2_file_name: str = "config-4b.json"
config_2_path: Path = current_dir_path / config_2_file_name

with config_1_path.open() as f1:
    config_1 = json.load(f1)

with config_2_path.open() as f2:
    config_2 = json.load(f2)

In [ ]:
import sys
sys.path.append(str(current_dir_path.resolve().parent.parent))
from llm_application.azure_information_assistant_accelerator.wrapper import RAG_from_scratch

rag_chain_1 = RAG_from_scratch(config_data=config_1)
rag_chain_2 = RAG_from_scratch(config_data=config_2)

In [ ]:
from kjr_llm.app import App
from kjr_llm.targets import CustomTarget

app = App(app_name="RAG_Application", reset_database=True)

target_1: CustomTarget = CustomTarget(rag_chain_1)
target_2: CustomTarget = CustomTarget(rag_chain_2)

In [ ]:
from kjr_llm.prompts import PromptSet

prompts_path_1: Path = current_dir_path / 'relevance-4a.json'
prompts_1 = PromptSet.from_json_file(prompts_path_1)

prompts_path_2: Path = current_dir_path / 'relevance-4b.json'
prompts_2 = PromptSet.from_json_file(prompts_path_2)

In [ ]:
from kjr_llm.metrics import AnswerRelevance, Groundedness
from trulens.core.schema import Select

context_path = Select.Record.app.retrieve.rets[:]
metrics = [
    AnswerRelevance().openai,
    Groundedness(context_path).openai
]

In [ ]:
from kjr_llm.tests import TestSet
from kjr_llm.provider import OpenAIProvider

test_set = TestSet(prompts_1, metrics, name="Exercise-4b",
                   default_provider=OpenAIProvider(model_name="gpt-3.5-turbo"))

targets = [target_1, target_2]

for i, target in enumerate(targets):
    test_set.evaluate(target, app_id=f"Exercise4b-app{i}")

app.run_dashboard()